In [ ]:
import os

from simulation_environment import K8sAutoscalerEnv
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
env = K8sAutoscalerEnv(deployment_name="nginx-deployment", iteration=3, verbose=True, action_step=50, timeout=30)

In [ ]:
env.observation_space.sample()

In [ ]:
episodes = 1
for episode in range(1, episodes + 1):
    env.reset()
    terminated = False
    truncated = False
    score = 0

    while not terminated and not truncated:
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        score += reward
        if env.verbose:
            print(f"Action: {action}, Reward: {reward}, Terminated: {terminated}, Truncated: {truncated}")
            print(f"CPU Usage: {obs[0]}, Memory Usage: {obs[1]}, Replicas: {obs[2]}")
            print("===========================")

    print(f"Episode: {episode}  Score {score}")
env.close()


In [ ]:
env = K8sAutoscalerEnv(
    deployment_name="nginx-deployment",
    iteration=3,
    verbose=True,
    action_step=5,
    timeout=30,
)
log_path = os.path.join("training", "logs")
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(3)

In [ ]:
simulation_path = os.path.join("training", "model", "Shower_Model_PPO")

In [ ]:
model.save(simulation_path)

In [ ]:
del model

In [ ]:
model = PPO.load(simulation_path, env)


In [ ]:
# evaluate_policy(model, env, 10)

In [ ]:
from kubernetes import config, client

In [ ]:
config.load_kube_config()


In [ ]:
namespace = "default"
deployment_name = "nginx-deployment"

In [ ]:
api = client.CustomObjectsApi()

# Replace 'default' with your namespace
metric_data = api.list_namespaced_custom_object(
    group="metrics.k8s.io", version="v1beta1", namespace="default", plural="pods"
)


In [ ]:
for item in metric_data["items"]:
    name = item["metadata"]["name"]
    for ctr in item["containers"]:
        cpu = ctr["usage"]["cpu"]
        memory = ctr["usage"]["memory"]
        print(f"{name} ▶️ CPU: {cpu}, MEM: {memory}")


In [ ]:
cluster = client.AppsV1Api()

In [ ]:
deployment = cluster.read_namespaced_deployment(
    name=deployment_name, namespace=namespace
)
